In [2]:
import os,cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

from keras import backend as K
K.set_image_dim_ordering('tf')

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam,adadelta

C:\Users\Nipuni\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [3]:
import os.path
from keras.models import load_model

USE_SKLEARN_PREPROCESSING=False

from sklearn import preprocessing

def image_to_feature_vector(image,size=(128,128)):
    #resize the image to a fixed size , then flttern the image into a list of raw pixels intensities
    return cv2.resize(image,size).flatten()

if USE_SKLEARN_PREPROCESSING:
    img_data=img_data_scaled
    
PATH = os.getcwd()
# Define data path
data_path = PATH + '\Original\DATA SET'
data_dir_list = os.listdir(data_path)

img_rows=128
img_colms=128
num_channels=3
num_epoch=6

img_data_list=[]
for dataset in data_dir_list:
    print(data_dir_list)
    img_list=os.listdir(data_path+'/'+dataset)
    print('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path+'/'+dataset+'/'+img)
            
        input_img_flatten=image_to_feature_vector(input_img,(128,128))
            
        img_data_list.append(input_img_flatten)

img_data=np.array(img_data_list)
img_data=img_data.astype('float')
print('Image Data',img_data.shape)

if num_channels==1:
    if K.image_dim_ordering()=='th':
        img_data=np.expand_dims(img_data,axis=1)
        print('Image Data BnW',img_data.shape)
    else:
        img_data=np.expand_dims(img_data,axis=4)
        print('Image Data BnW',img_data.shape)

else:
    if K.image_dim_ordering()=='th':
        img_data=np.rollaxis(img_data,3,1)
        print('Image Data RGB',img_data.shape)

image_data_scaled=preprocessing.scale(img_data)
print("Image Data Scaled" , image_data_scaled)

if K.image_dim_ordering()=='th':
    image_data_scaled=image_data_scaled.reshape(img_data.shape[0],num_channels,img_rows,img_colms)
    print('Image Data Scaled BnW',image_data_scaled.shape)
else:
    image_data_scaled=image_data_scaled.reshape(img_data.shape[0],img_rows,img_colms,num_channels)
    print('Image Data Scaled RGB',image_data_scaled.shape)

#Define classes

num_classes=2

num_samples=img_data.shape[0]
labels=np.ones((num_samples,),dtype='int64')

labels[0:113]=0
labels[113:]=1

names=['negative','positive']

#convert class labels to on-hot encoding
Y=np_utils.to_categorical(labels,num_classes)

x,y=shuffle(img_data,Y,random_state=2)

X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=4)

X_train = X_train.reshape(X_train.shape[0], img_colms, img_rows, -1)
X_test = X_test.reshape(X_test.shape[0], img_colms, img_rows, -1)

input_shape=(img_colms, img_rows, 1)

# Defining the model
input_shape=img_data[0].shape

model = Sequential()

model.add(Conv2D(32, (3,3),border_mode='same',input_shape=(128,128,3)))
model.add(Activation('relu'))

model.add(Conv2D(32,( 3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2),dim_ordering='th'))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3),dim_ordering='th'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2),dim_ordering='th'))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512)) #no of hidden layers
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']) 

# Viewing model_configuration

model.summary()
model.get_config()
model.layers[0].get_config()
model.layers[0].input_shape
model.layers[0].output_shape
model.layers[0].get_weights()
np.shape(model.layers[0].get_weights()[0])
model.layers[0].trainable

# Training

if os.path.isfile('Hand_Gesture_Recognition.h5') == False:
    hist = model.fit(X_train, y_train, batch_size=150, epochs=num_epoch, verbose=1, validation_data=(X_test, y_test))
    model.save('Hand_Gesture_Recognition.h5')
else:
    hist=load_model('Hand_Gesture_Recognition.h5')

# Evaluating the model

score = model.evaluate(X_test, y_test, batch_size=150 , verbose=0) #batch_size=16
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

test_image = X_test[0:1]

#print(model.predict(test_image))
#print(model.predict_classes(test_image))
y_test[0:1]



['Negative', 'Positive']
Loaded the images of dataset-Negative

['Negative', 'Positive']
Loaded the images of dataset-Positive

Image Data (234, 49152)
Image Data Scaled [[-0.06999187 -0.06604903 -0.03929729 ..., -0.16223969 -0.11872931
  -0.07215629]
 [-0.35696729 -0.39779527 -0.38733662 ...,  0.39891226  0.28597075
   0.34767629]
 [-0.09049011 -0.02702006 -0.05761515 ..., -0.11907416 -0.24013933
  -0.4189745 ]
 ..., 
 [-0.60294623 -0.78808498 -0.73537595 ...,  2.79459946  2.91652113
   2.84841817]
 [-0.02899538 -0.18313594 -0.13088659 ..., -1.06871593 -1.04953945
  -0.96658222]
 [-0.50045501 -0.45633873 -0.3507009  ...,  0.33416396  0.42761577
   0.45719783]]
Image Data Scaled RGB (234, 128, 128, 3)


C:\Users\Nipuni\Anaconda3\lib\site-packages\ipykernel_launcher.py:93: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(128, 128,..., padding="same")`
C:\Users\Nipuni\Anaconda3\lib\site-packages\ipykernel_launcher.py:99: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
C:\Users\Nipuni\Anaconda3\lib\site-packages\ipykernel_launcher.py:102: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), data_format="channels_first")`
C:\Users\Nipuni\Anaconda3\lib\site-packages\ipykernel_launcher.py:105: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 126, 126, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 63, 16)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 126, 63, 16)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 61, 14)        72640     
__________

array([[ 0.,  1.]])

In [4]:
# Testing a new image
def Get_Reult_Of_HG(filepath):

    test_image = cv2.imread(filepath)
    #test_image=cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
    test_image=cv2.resize(test_image,(128,128))
    test_image = np.array(test_image)
    test_image = test_image.astype('float32')
    test_image /= 255
    print (test_image.shape)

    if num_channels==1:
        if K.image_dim_ordering()=='th':
            test_image= np.expand_dims(test_image, axis=1)
            print (test_image.shape)

        else:
            test_image= np.expand_dims(test_image, axis=4) 
            print (test_image.shape)

    else:
        if K.image_dim_ordering()=='th':
            test_image=np.rollaxis(test_image,3,1)
            print (test_image.shape)
        else:
            test_image= np.expand_dims(test_image, axis=0)
            print (test_image.shape)

    # Predicting the test image
    print(model.predict(test_image))
    predict_class=model.predict_classes(test_image)
    print(predict_class)

    result=''

    if predict_class==[1]:
        print('Result :',predict_class,'=> Positive Hand Gesture')
        result='Positive Hand Gesture'

    elif predict_class==[0]:
        print('Result :',predict_class,'=> Negative Hand Gesture')
        result='Negative Hand Gesture'
        
    return result



In [5]:
import cv2

In [6]:
def capture_object():

    cam=cv2.VideoCapture(0)

    cv2.namedWindow("test")

    img_counter=0

    while True:
        ret, frame=cam.read()
        cv2.imshow("test",frame)
        if not ret:
            break
        k=cv2.waitKey(1)

        if k%256 == 27:
            #ESC pressed
            print("Escape hit, closing...")
            break
        elif k%256 == 32:
            #SPACE pressed
            img_name="opencv_frame_{}.png".format(img_counter)

            print("{} written!".format(img_name))

            img_counter +=1

    cam.release()

    cv2.destroyAllWindows()

In [7]:
#Capture image of the object from the camera
def capture_Object(output):
    
    #Checking whether the captured hand gesture is a valid hand gesture to track the object
    if output=='Positive Hand Gesture':
        
        capture_object()
                    
        print("Taking image...")

    else :
        print('Hand Gesture is not suitable for tracking the object')
        
capture_Object(Get_Reult_Of_HG(filepath='Original/Test/hand_gesture_0.png'))

(128, 128, 3)
(1, 128, 128, 3)
[[ 0.43320486  0.56679505]]
1/1 [==============================] - 0s
[1]
Result : [1] => Positive Hand Gesture


error: C:\projects\opencv-python\opencv\modules\highgui\src\window.cpp:325: error: (-215) size.width>0 && size.height>0 in function cv::imshow


In [ ]:
import cv2
import numpy as np
cap = cv2.VideoCapture(0)
while( cap.isOpened() ) :
    ret,img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    ret,thresh1 = cv2.threshold(blur,70,255,(cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU))
  
    contours, hierarchy = cv2.findContours(thresh1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    drawing = np.zeros(img.shape,np.uint8)

    max_area=0
   
    for i in range(len(contours)):
            cnt=contours[i]
            area = cv2.contourArea(cnt)
            if(area>max_area):
                max_area=area
                ci=i
    cnt=contours[ci]
    hull = cv2.convexHull(cnt)
    moments = cv2.moments(cnt)
    if moments['m00']!=0:
                cx = int(moments['m10']/moments['m00']) # cx = M10/M00
                cy = int(moments['m01']/moments['m00']) # cy = M01/M00
              
    centr=(cx,cy)       
    cv2.circle(img,centr,5,[0,0,255],2)       
    cv2.drawContours(drawing,[cnt],0,(0,255,0),2) 
    cv2.drawContours(drawing,[hull],0,(0,0,255),2) 
          
    cnt = cv2.approxPolyDP(cnt,0.01*cv2.arcLength(cnt,True),True)
    hull = cv2.convexHull(cnt,returnPoints = False)
    
    if(1):
               defects = cv2.convexityDefects(cnt,hull)
               mind=0
               maxd=0
               for i in range(defects.shape[0]):
                    s,e,f,d = defects[i,0]
                    start = tuple(cnt[s][0])
                    end = tuple(cnt[e][0])
                    far = tuple(cnt[f][0])
                    dist = cv2.pointPolygonTest(cnt,centr,True)
                    cv2.line(img,start,end,[0,255,0],2)
                    
                    cv2.circle(img,far,5,[0,0,255],-1)
               print(i)
               i=0
    cv2.imshow('output',drawing)
    cv2.imshow('input',img)
                
    k = cv2.waitKey(10)
    if k == 27:
        break